In [1]:
using Distributions
using ReactiveMP
using Rx
using BenchmarkTools

import Base: show

┌ Info: Precompiling Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1273
┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1273
┌ Warning: Package ReactiveMP does not have Rx in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rx as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rx into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.


In [2]:
function createSubgraph(index::Int)
    noise_add = addition_node("[$index] noise_add", StochasticMessage{Normal{Float64}}, StochasticMessage{Normal{Float64}}, DeterministicMessage);
    noise_node     = gaussian_mean_variance("[$index] noise_node");
    noise_mean     = constant_variable("[$index] noise_mean", 0.0, noise_node.mean);
    noise_variance = constant_variable("[$index] noise_variance", 1.0, noise_node.variance);
    
    tmp = random_variable("[$index] tmp", noise_node.value, noise_add.in2)
    yn = observed_variable("[$index] yn", noise_add.out)
    
    return (yn, noise_add)
end

function createGraph(size::Int)
    gmv_0   = gaussian_mean_variance("[0] gmv");
    gmv_0_m = estimated_variable("[0] gmv_m", gmv_0.mean);
    gmv_0_v = estimated_variable("[0] gmv_v", gmv_0.variance);
    
    add_c_0 = addition_node("[0] add_c", StochasticMessage{Normal{Float64}}, DeterministicMessage, StochasticMessage{Normal{Float64}});
    x0 = random_variable("[0] x", gmv_0.value, add_c_0.in1)
    c0 = constant_variable("[0] c", 1.0, add_c_0.in2)
    
    index = 1
    
    prev_add_c = add_c_0
    
    xs = Vector{RandomVariable}(undef, size)
    ys = Vector{ObservedVariable}(undef, size)
    
    while index < size
        equality_n    = equality_ioo("[$index] equality")
        yn, noise_add = createSubgraph(index)
        
        xn  = random_variable("[$index] xn", prev_add_c.out, equality_n.in1)
        xn_ = random_variable("[$index] xn_", equality_n.out1, noise_add.in1)
        
        add_c = addition_node("[$index] add_c", StochasticMessage{Normal{Float64}}, DeterministicMessage, StochasticMessage{Normal{Float64}});
        
        xn__ = random_variable("[$index] xn__", equality_n.out2, add_c.in1)
        cn   = constant_variable("[$index] cn", 1.0, add_c.in2)
        
        xs[index] = xn
        ys[index] = yn
        
        prev_add_c = add_c
        
        index += 1
    end
    
    last_noise_add = addition_node("[last] noise_add", StochasticMessage{Normal{Float64}}, StochasticMessage{Normal{Float64}}, DeterministicMessage);
    x_last = random_variable("[last] x", prev_add_c.out, last_noise_add.in1)
    
    last_noise_node     = gaussian_mean_variance("[last] noise_node");
    last_noise_mean     = constant_variable("[last] noise_mean", 0.0, last_noise_node.mean);
    last_noise_variance = constant_variable("[last] noise_variance", 1.0, last_noise_node.variance);
    
    z      = random_variable("[last] z", last_noise_node.value, last_noise_add.in2)
    y_last = observed_variable("[last] y", last_noise_add.out)
    
    xs[size] = x_last
    ys[size] = y_last
    
    return (xs, ys, gmv_0_m, gmv_0_v)
end

createGraph (generic function with 1 method)

In [3]:
function smoothing_replay(N, data)
    xs, ys, prior_m, prior_v = createGraph(N);
    
    keepa = keep(AbstractMessage)
    actor = sync(keepa)
    
    subscribe!(inference(xs[1]) |> take(1), actor)
    subscribe!(inference(xs[N]) |> take(1), actor)
    
    update!(prior_m, 0.0)
    update!(prior_v, 1000.0)
    for i in 1:N
       update!(ys[i], data[i])
    end
    
    wait(actor)
    
    for i in 2:N-1
        inner_synced = sync(keepa)
        subscribe!(inference(xs[i]) |> take(1), inner_synced)
        wait(inner_synced)
    end
    
    return keepa.values
end

smoothing_replay (generic function with 1 method)

In [4]:
N = 300
data = collect(1:N) + rand(Normal(0.0, 1.0), N);

In [7]:
@btime v = smoothing_replay($N, $data);

  2.974 s (19871656 allocations: 916.96 MiB)
